In [1]:
# define function to download data
def load_data(url, filename):
    import urllib.request
    from zipfile import ZipFile
    
    response = urllib.request.urlretrieve(
        url,filename)
    #unzip
    with ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall()

In [2]:
# define fuction to read data from xml file
def import_data(filename):
    import xml.etree.ElementTree as et 
    xtree = et.parse(filename)
    xroot = xtree.getroot()
    
    return xroot;

In [3]:
# define function to create distance matrix
def dist_matrix(cities, xroot):
    #create distance matrix
    import numpy as np
    distance = np.zeros((cities,cities))
    
    #import data
    import xml.etree.ElementTree as et
    from_node = 0
    for child in xroot.iter('vertex'):
        for child1 in child:
            dist = float(child1.attrib.get('cost'))
            to_node = int(child1.text)
            distance[from_node, to_node] = dist

        from_node += 1
    
    max_distance = np.nanmax(distance)
    for i in range(cities):#
        distance[i,i] = max_distance*10 #very large number for distance to itself => no revisited 

    return distance 

In [4]:
# define function to calculate the length of the tour 
def totalcost(tour, distance):
    length = 0
    for i in range(len(tour)-1):
        length += distance[tour[i]][tour[i+1]]
    return length

In [5]:
# starting tour option

In [6]:
# define function to execute random tour
def random_tour(cities, distance):
    import numpy as np
    
    temp = [x for x in range(1,cities)]
    np.random.shuffle(temp)
    tour = [0] + temp + [0]
    length = totalcost(tour, distance)
    
    return (tour, length)

In [7]:
# define function to execute nearest neighbor        
def nearest_neighbor(cities, distance):
    import numpy as np

    position = 0
    tour = [0]
    length = 0
    for i in range(cities-1):
        nn = 0
        nd = np.nanmax(distance)+1
        for j in range(cities):  
            if (j not in tour) and (distance[position,j]<nd) and (i!=j):
                nd = distance[position,j]
                nn = j
        tour.append(nn)
        length = length + nd
        position = nn
    tour.append(0)
    length = length + distance[position,0]
    
    return (tour, length)

In [8]:
# define function to execute successive insertion
def succ_insert(cities, distance):
    import numpy as np

    tour = [0, 0] #start and end at node 0
    length = 0 #total length of current tour
    
    for i in range(1, cities) : #add nodes in order of 1 -> nodes-1
        length_change = np.zeros(len(tour) - 1) #list of change in length for each possible position to insert new node

        for position in range(len(tour) - 1): #iterate through possible positions in current tour
            temp_tour = tour[: position+1] + [i] + tour[position+1 :]

            #calculate the change in length between current tour and temp tour
            pre_ins_node = tour[position]
            nex_ins_node = tour[position + 1]
            if pre_ins_node == nex_ins_node:
                length_change[position] = distance[pre_ins_node][i] + distance[i][nex_ins_node]
            else:
                length_change[position] = distance[pre_ins_node][i] + distance[i][nex_ins_node] - distance[pre_ins_node][nex_ins_node]

        min_position = np.argmin(length_change) #get the position which create the shortest length change
        length += length_change[min_position]
        tour = tour[: min_position+1] + [i] + tour[min_position+1 :]
        
    return (tour, length)

In [9]:
#neighborhood definition options

In [10]:
#exchange
def random_exchange(tour, distance):
    [i,j] = np.random.choice(range(1,len(tour)-1), size = 2, replace = False) #start = end = depot
    
    delta = distance[tour[i-1],tour[i]] + distance[tour[i],tour[i+1]] + distance[tour[j-1],tour[j]] + distance[tour[j],tour[j+1]] - distance[tour[i-1],tour[j]] - distance[tour[j],tour[i+1]] - distance[tour[j-1],tour[i]] - distance[tour[i],tour[j+1]]
    #improvement = positive delta
    
    #exchange node i and j
    new_tour = copy.deepcopy(tour)
    new_tour[i] = tour[j]
    new_tour[j] = tour[i]
    
    return(new_tour, delta)


In [11]:
# define function to swap 2 edges of nodes with index m and n (starting index 0)  
def tour_swap (tour, m, n):
    new_tour = copy.deepcopy(tour)
    new_tour[m:n+1] = reversed(new_tour[m:n+1])
    return new_tour

In [12]:
#2opt
def random_2opt(tour, distance):
    i = np.random.randint(1,len(tour)-2)
    j = np.random.randint(i+1,len(tour)-1)
    
    new_tour = tour_swap(tour, i, j)
    delta = totalcost(tour[(i-1):(j+2)], distance) - totalcost(new_tour[(i-1):(j+2)], distance) #improvement = positive delta
    
    return(new_tour, delta)

In [22]:
#main function
def SA (tour, length, distance, initial=0.01, cool_factor=0.95, eval_times=10000, neighbor = 'two_opt', 
        TimeLimit = None, non_improvement = None, max_iteration = None, min_temp = None):

    best = length
    besttour = tour
    best_iter = 0
    temperature = initial*length
    print(tour)
    print(best)
    
    #time limit condition
    from time import process_time 
    start_time = process_time()
     
    #max iteration condition
    iteration = 0
    non_iter = 0
    
    terminated = False
    while not terminated: #termination condition 
        
        improvement = False
        iteration += 1 
        
        #evaluation at given alpha
        for k in range(eval_times):
            next_tour = []
            if neighbor == 'exchange':
                (next_tour, delta) = random_exchange(tour, distance) #end = depot
            elif neighbor == 'two_opt':
                (next_tour, delta) = random_2opt(tour, distance)
            else:
                print('Invalid neighborhood definition')
                break
                
            if delta > 0 or (np.random.random() < math.exp(delta/temperature)): 
                #print('****', length, delta)
                length = length - delta
                tour = next_tour
            if length < best:
                best = length
                besttour=tour
                best_iter = iteration
                improvement = True
                #print(best, '   ', temperature)
               
        temperature = cool_factor*temperature

        if (TimeLimit is not None): #time elapsed termination
            runtime = process_time() - start_time
            if (runtime >= TimeLimit): 
                print('Reach TimeLimit')
                terminated = True
                
        elif (non_improvement is not None): #max iteration without new best solution
            if improvement: 
                non_iter = 0
            else: non_iter += 1
            if non_iter >= non_improvement:
                print('Reach max number of iteration without new best solution')
                terminated = True
        
        elif (max_iteration is not None):
            if iteration >= max_iteration:
                print('Reach max number of iteration')
                terminated = True
                
        elif (min_temp is not None):
            if  temperature < min_temp : 
                print('Reach min temperature')
                terminated = True
                
        else: 
            if  temperature < (0.95**100) * initial*length: #default option 
                print('Reach default min temperature')
                terminated = True
    
    print("Terminate")
    print(besttour)
    print(best)
    
    return (best, iteration, best_iter)

In [14]:
import numpy as np
import pandas as pd
import math
import random
import copy

In [15]:
# 120 cities
cities = 120

di = pd.read_excel("gr120.xlsx",sheet_name="DistanceMatrix")
distance=di.values

for i in range(cities):
    distance[i,i] = 10000

In [16]:
# Medium size data set: # of nodes = 152
url1 = 'http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/XML-TSPLIB/instances/pr152.xml.zip'
zip1 = 'pr152.xml.zip'
file1 = 'pr152.xml'
cities1 = 152

load_data(url1, zip1)
xroot1 = import_data(file1)
distance1 = dist_matrix(cities1, xroot1)

In [17]:
# Larger size data set: # of nodes = 264
url2 = 'http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/XML-TSPLIB/instances/pr264.xml.zip'
zip2 = 'pr264.xml.zip'
file2 = 'pr264.xml'
cities2 = 264

load_data(url2, zip2)
xroot2 = import_data(file2)
distance2 = dist_matrix(cities2, xroot2)

In [18]:
dataset = ((cities, distance), (cities1, distance1), (cities2, distance2))
index = ['Dataset 1', 'Dataset 2', 'Dataset 3']

**Notation**  
Dataset 1 = Example with 120 cities  
Dataset 2 = Example with 152 cities  
Dataset 3 = Example with 264 cities  

### Comparision between Starting solution  
Run 3 starting solution strategies: random, nearest neighbor, successive insertion with 3 datasets, the other parameters are default (initial = 0.01, cooling factor = 0.95, number of evaluation each iteration = 10000, neighborhood definition = 'two_opt'). The termination condition is set as TimeLimit = 60 seconds per dataset per strategy. 

In [23]:
import pandas as pd
value = pd.DataFrame(index = index, columns = ['random', 'nearest neighbor', 'successive insertion'])

#random starting tour
for i in range(3):
    (start_tour, start_length) = random_tour(dataset[i][0], dataset[i][1])
    result = SA (start_tour, start_length, dataset[i][1], initial=0.01, cool_factor=0.95, eval_times=10000, neighbor = 'two_opt', TimeLimit = 60)
    value.iloc[i,0] = result

#nearest neighbor
for i in range(3):
    (start_tour, start_length) = nearest_neighbor(dataset[i][0], dataset[i][1])
    result = SA (start_tour, start_length, dataset[i][1], initial=0.01, cool_factor=0.95, eval_times=10000, neighbor = 'two_opt', TimeLimit = 60)
    value.iloc[i,1] = result

#successive insertion
for i in range(3):
    (start_tour, start_length) = succ_insert(dataset[i][0], dataset[i][1])
    result = SA (start_tour, start_length, dataset[i][1], initial=0.01, cool_factor=0.95, eval_times=10000, neighbor = 'two_opt', TimeLimit = 60)
    value.iloc[i,2] = result


[0, 10, 8, 12, 14, 103, 52, 41, 13, 36, 6, 109, 62, 61, 46, 39, 116, 25, 101, 11, 58, 63, 95, 75, 45, 70, 50, 102, 22, 43, 48, 30, 40, 105, 21, 115, 16, 37, 72, 19, 34, 108, 119, 57, 31, 87, 27, 3, 15, 96, 85, 114, 55, 65, 104, 80, 7, 68, 73, 118, 29, 110, 47, 93, 98, 76, 28, 60, 51, 67, 78, 26, 42, 35, 20, 71, 84, 88, 112, 64, 106, 53, 9, 56, 91, 94, 17, 100, 82, 83, 5, 92, 74, 66, 117, 107, 44, 32, 86, 111, 54, 97, 4, 79, 18, 59, 81, 38, 49, 69, 89, 113, 23, 2, 99, 77, 90, 33, 24, 1, 0]
54019
Reach TimeLimit
Terminate
[0, 28, 15, 59, 23, 110, 95, 89, 70, 46, 88, 47, 101, 100, 109, 111, 105, 103, 61, 36, 98, 9, 34, 83, 35, 5, 54, 53, 7, 69, 115, 25, 3, 33, 71, 39, 104, 60, 119, 31, 91, 85, 13, 86, 73, 74, 49, 45, 97, 41, 16, 48, 117, 12, 50, 22, 8, 10, 114, 1, 20, 108, 87, 63, 92, 52, 26, 4, 62, 76, 94, 96, 11, 38, 82, 66, 56, 72, 113, 79, 81, 2, 118, 102, 37, 6, 55, 40, 43, 106, 19, 112, 68, 64, 57, 78, 51, 32, 99, 90, 67, 42, 107, 24, 18, 17, 84, 116, 65, 30, 21, 80, 93, 77, 44, 27,

Reach TimeLimit
Terminate
[0, 39, 40, 38, 37, 36, 60, 59, 120, 119, 118, 117, 116, 115, 62, 61, 35, 34, 33, 32, 31, 30, 64, 63, 114, 113, 112, 111, 110, 109, 66, 65, 29, 28, 27, 26, 25, 24, 68, 67, 108, 107, 106, 105, 104, 103, 70, 69, 23, 22, 21, 20, 19, 18, 72, 71, 102, 101, 100, 99, 98, 97, 74, 73, 17, 16, 15, 14, 13, 12, 76, 75, 96, 95, 94, 93, 92, 91, 78, 77, 11, 10, 9, 8, 7, 6, 80, 79, 90, 89, 88, 87, 86, 85, 82, 81, 5, 4, 3, 2, 1, 83, 84, 131, 130, 129, 128, 41, 42, 43, 44, 45, 47, 46, 48, 56, 57, 121, 122, 123, 124, 125, 126, 54, 55, 49, 50, 51, 52, 53, 127, 58, 145, 146, 147, 144, 164, 163, 162, 224, 223, 237, 236, 239, 238, 222, 221, 167, 166, 165, 143, 142, 170, 169, 168, 220, 219, 241, 240, 243, 242, 218, 217, 173, 172, 171, 141, 140, 174, 175, 176, 215, 216, 244, 245, 246, 247, 213, 214, 177, 178, 179, 139, 138, 180, 181, 182, 211, 212, 248, 249, 250, 251, 209, 210, 183, 184, 185, 137, 136, 186, 187, 188, 207, 208, 252, 253, 254, 255, 205, 206, 189, 190, 191, 135, 134, 194

Then we test 2 times and compare the results obtained from each strategy for each dataset. Each time, the result includes 3 information, which are (objective value, total number of iteration, run before that best solution is found).

In [21]:
value1=value
print(value1)

                                 random             nearest neighbor  \
Dataset 1                (8037, 61, 61)               (7095, 60, 60)   
Dataset 2  (139474.69752937893, 53, 51)  (79435.48153405348, 47, 47)   
Dataset 3   (447623.8695958158, 32, 32)     (58022.856841092, 31, 0)   

                 successive insertion  
Dataset 1            (7105.0, 61, 61)  
Dataset 2  (75694.2438165863, 51, 51)  
Dataset 3  (57260.05492393449, 30, 0)  


In [25]:
value2=value
print(value2)

                                 random             nearest neighbor  \
Dataset 1                (7993, 63, 63)               (7029, 65, 54)   
Dataset 2  (156927.70357077933, 50, 50)  (74686.59879259461, 53, 53)   
Dataset 3   (530284.3518024081, 28, 28)     (58022.856841092, 33, 0)   

                  successive insertion  
Dataset 1             (7130.0, 66, 59)  
Dataset 2  (75544.55386645159, 56, 50)  
Dataset 3   (57260.05492393449, 32, 0)  


#### Comment:
The performance of Nearest neighbor and Successive insertion is approximately the same. The Successive insertion works slightly worse than Nearest neighbor in dataset 1. In dataset 2, there is a mixed result, 1 test Successive insertion finds better solution and 1 test find worse one. In both tests, Successive insertion realizes tsolution quicker (fewer iterations needed) than Nearest neighbor.
However, the performance of Random starting tour is very different from the other two strategies. Random tour works worse in all datasets and in both tests, especially in large datasets. In general, the random starting tour is an unstable strategy. Therefore, it is safer to use heuristic starting tour than random one.

### **Comparision between Neighborhood definition**   
Run 3 Neighborhood definitions: 2-opt and random_exchange with 3 datasets. The starting solution is chosen as successive insertion, based on the previous comparision. The other parameters are default (initial = 0.01, cooling factor = 0.95, number of evaluation each iteration = 10000). The termination condition is set as TimeLimit = 60 seconds per dataset per definition.

In [26]:
start = []
for i in range(3):
    (start_tour, start_length) = succ_insert(dataset[i][0], dataset[i][1])
    start.append((start_tour, start_length))


In [30]:
import pandas as pd
value = pd.DataFrame(index = index, columns = ['two opt', 'exchange'])

#two opt
for i in range(3):
    result = SA (start[i][0], start[i][1], dataset[i][1], initial=0.01, cool_factor=0.95, eval_times=10000, neighbor = 'two_opt', TimeLimit = 60)
    value.iloc[i,0] = result

#exchange
for i in range(3):
    result = SA (start[i][0], start[i][1], dataset[i][1], initial=0.01, cool_factor=0.95, eval_times=10000, neighbor = 'exchange', TimeLimit = 60)
    value.iloc[i,1] = result


[0, 60, 15, 59, 23, 110, 95, 89, 53, 7, 69, 115, 33, 25, 70, 46, 88, 111, 35, 34, 9, 98, 105, 66, 36, 61, 103, 83, 5, 54, 47, 109, 100, 101, 2, 81, 79, 52, 26, 4, 62, 113, 72, 56, 82, 38, 94, 96, 11, 76, 63, 87, 108, 20, 92, 1, 114, 10, 50, 12, 22, 8, 102, 118, 3, 39, 104, 71, 37, 6, 55, 40, 41, 97, 117, 48, 16, 49, 45, 19, 106, 112, 68, 64, 42, 67, 90, 78, 57, 99, 32, 51, 107, 24, 18, 17, 84, 116, 30, 65, 80, 93, 21, 85, 74, 43, 86, 73, 13, 77, 44, 27, 91, 31, 58, 14, 29, 119, 28, 75, 0]
7779.0
Reach TimeLimit
Terminate
[0, 60, 15, 59, 23, 110, 95, 89, 53, 7, 69, 115, 33, 3, 25, 70, 46, 47, 101, 100, 109, 111, 103, 35, 88, 54, 5, 83, 34, 9, 98, 61, 36, 66, 105, 113, 72, 56, 82, 38, 62, 76, 94, 11, 96, 87, 108, 20, 114, 1, 92, 63, 52, 4, 26, 79, 81, 2, 118, 102, 8, 22, 10, 50, 12, 48, 117, 16, 97, 41, 40, 55, 6, 37, 71, 39, 104, 73, 86, 13, 74, 43, 49, 45, 19, 106, 112, 68, 64, 67, 90, 57, 99, 32, 51, 78, 42, 107, 24, 18, 17, 84, 116, 30, 65, 21, 80, 93, 85, 77, 44, 27, 91, 31, 119, 28

Reach TimeLimit
Terminate
[0, 38, 33, 32, 31, 27, 25, 26, 21, 19, 20, 15, 13, 14, 9, 7, 8, 3, 2, 1, 4, 5, 81, 83, 84, 87, 86, 85, 82, 6, 80, 79, 90, 89, 88, 93, 92, 91, 78, 77, 11, 10, 12, 76, 75, 96, 95, 94, 131, 132, 198, 133, 195, 196, 197, 200, 199, 259, 258, 202, 201, 194, 134, 135, 191, 190, 193, 192, 204, 203, 257, 256, 205, 206, 189, 255, 254, 260, 261, 262, 253, 252, 208, 207, 188, 187, 186, 136, 137, 185, 184, 183, 210, 209, 251, 250, 249, 248, 212, 211, 182, 181, 180, 138, 139, 179, 178, 177, 214, 213, 247, 246, 245, 244, 216, 215, 176, 175, 174, 140, 141, 171, 172, 173, 217, 218, 242, 243, 240, 241, 219, 220, 170, 142, 143, 165, 166, 169, 168, 167, 221, 222, 238, 239, 263, 236, 237, 223, 224, 162, 163, 164, 144, 146, 147, 145, 161, 160, 225, 159, 158, 157, 148, 154, 155, 156, 226, 227, 234, 235, 232, 231, 233, 228, 229, 152, 151, 230, 150, 153, 149, 130, 129, 99, 98, 97, 74, 73, 17, 16, 18, 72, 71, 102, 101, 100, 105, 104, 103, 70, 69, 23, 22, 24, 68, 67, 108, 107, 106, 111

Then we test 2 times and compare the results obtained from each definition for each dataset. Each time, the result includes 3 information, which are (objective value, total number of iteration run, the iteration that best solution is found).

In [29]:
value3=value
print(value3)

                               two opt                    exchange
Dataset 1             (6992.0, 64, 56)             (7779.0, 44, 0)
Dataset 2  (74974.69599370731, 52, 51)  (91334.72161657987, 40, 0)
Dataset 3   (57260.05492393449, 30, 0)  (57260.05492393449, 29, 0)


In [33]:
value4=value
print(value4)

                               two opt                    exchange
Dataset 1             (7004.0, 63, 47)             (7753.0, 45, 1)
Dataset 2  (75215.47128658224, 54, 54)  (91334.72161657987, 40, 0)
Dataset 3   (57260.05492393449, 32, 0)  (57260.05492393449, 29, 0)


#### **Comment:**  
The performance of 2-opt is better than that of Exchange in dataset 1 and 2 in both two tests. In dataset 3, because of large solution space, both approaches seem to stuck in the starting solution and may need more time to jump out. Noticeably, for exchange definition, there nearly isn't any improvement from the starting solution in all three datasets (except for the dataset 1 in second test), suggessting that this approach in general may need longer time to start showing effective.

### **Comparision between Cooling schedules: Initial temperature**  
Run 3 settings of Initial temperature to: 0.1, 0.01 and 0.001 (\*starting_length) with 3 datasets. The starting solution, neighborhood definition are chosen as successive insertion and two_opt, based on the previous comparisions. The other parameters are default (cooling factor = 0.95, number of evaluation each iteration = 10000). This time, to focus more on the effect of temperature, the termination condition is set as the maximum number of iterations = 70, 60, 30 for dataset 1, 2, 3 respectively. So for each dataset, the range of temperature explored in each setting case will be solely depends on the initial setting and is the same in each time we test.

In [40]:
import pandas as pd
value = pd.DataFrame(index = index, columns = ['initial = 0.1', 'initial = 0.01', 'initial = 0.001'])
initial_set = [0.1, 0.01, 0.001]
iter_set = [70, 60, 30]

for i in range(3): #dataset 
    for j in range(3): #initial
        result = SA (start[i][0], start[i][1], dataset[i][1], initial=initial_set[j], 
                     cool_factor=0.95, eval_times=10000, neighbor = 'two_opt', max_iteration = iter_set[i])
        value.iloc[i,j] = result


[0, 60, 15, 59, 23, 110, 95, 89, 53, 7, 69, 115, 33, 25, 70, 46, 88, 111, 35, 34, 9, 98, 105, 66, 36, 61, 103, 83, 5, 54, 47, 109, 100, 101, 2, 81, 79, 52, 26, 4, 62, 113, 72, 56, 82, 38, 94, 96, 11, 76, 63, 87, 108, 20, 92, 1, 114, 10, 50, 12, 22, 8, 102, 118, 3, 39, 104, 71, 37, 6, 55, 40, 41, 97, 117, 48, 16, 49, 45, 19, 106, 112, 68, 64, 42, 67, 90, 78, 57, 99, 32, 51, 107, 24, 18, 17, 84, 116, 30, 65, 80, 93, 21, 85, 74, 43, 86, 73, 13, 77, 44, 27, 91, 31, 58, 14, 29, 119, 28, 75, 0]
7779.0
Reach max number of iteration
Terminate
[0, 60, 15, 59, 23, 110, 95, 89, 53, 7, 69, 115, 33, 25, 70, 46, 88, 111, 35, 34, 9, 98, 105, 66, 36, 61, 103, 83, 5, 54, 47, 109, 100, 101, 2, 81, 79, 52, 26, 4, 62, 113, 72, 56, 82, 38, 94, 96, 11, 76, 63, 87, 108, 20, 92, 1, 114, 10, 50, 12, 22, 8, 102, 118, 3, 39, 104, 71, 37, 6, 55, 40, 41, 97, 117, 48, 16, 49, 45, 19, 106, 112, 68, 64, 42, 67, 90, 78, 57, 99, 32, 51, 107, 24, 18, 17, 84, 116, 30, 65, 80, 93, 21, 85, 74, 43, 86, 73, 13, 77, 44, 27, 9

Reach max number of iteration
Terminate
[0, 38, 33, 32, 31, 27, 25, 26, 21, 19, 20, 15, 13, 14, 9, 7, 8, 3, 2, 1, 4, 5, 81, 83, 84, 87, 86, 85, 82, 6, 80, 79, 90, 89, 88, 93, 92, 91, 78, 77, 11, 10, 12, 76, 75, 96, 95, 94, 131, 132, 198, 133, 195, 196, 197, 200, 199, 259, 258, 202, 201, 194, 134, 135, 191, 190, 193, 192, 204, 203, 257, 256, 205, 206, 189, 255, 254, 260, 261, 262, 253, 252, 208, 207, 188, 187, 186, 136, 137, 185, 184, 183, 210, 209, 251, 250, 249, 248, 212, 211, 182, 181, 180, 138, 139, 179, 178, 177, 214, 213, 247, 246, 245, 244, 216, 215, 176, 175, 174, 140, 141, 171, 172, 173, 217, 218, 242, 243, 240, 241, 219, 220, 170, 142, 143, 165, 166, 169, 168, 167, 221, 222, 238, 239, 263, 236, 237, 223, 224, 162, 163, 164, 144, 146, 147, 145, 161, 160, 225, 159, 158, 157, 148, 154, 155, 156, 226, 227, 234, 235, 232, 231, 233, 228, 229, 152, 151, 230, 150, 153, 149, 130, 129, 99, 98, 97, 74, 73, 17, 16, 18, 72, 71, 102, 101, 100, 105, 104, 103, 70, 69, 23, 22, 24, 68, 67, 108,

In [39]:
value5=value
print(value5)

                        initial = 0.1               initial = 0.01  \
Dataset 1             (7779.0, 70, 0)             (7116.0, 70, 58)   
Dataset 2  (91334.72161657987, 60, 0)  (75088.71877153244, 60, 56)   
Dataset 3  (57260.05492393449, 30, 0)   (57260.05492393449, 30, 0)   

                       initial = 0.001  
Dataset 1             (7326.0, 70, 17)  
Dataset 2  (77488.94969419921, 60, 20)  
Dataset 3  (52228.21004257681, 30, 26)  


In [42]:
value6=value
print(value6)

                        initial = 0.1               initial = 0.01  \
Dataset 1             (7779.0, 70, 0)             (7127.0, 70, 64)   
Dataset 2  (91334.72161657987, 60, 0)  (76293.13032675172, 60, 58)   
Dataset 3  (57260.05492393449, 30, 0)   (57260.05492393449, 30, 0)   

                       initial = 0.001  
Dataset 1             (7352.0, 70, 23)  
Dataset 2   (74742.2880664401, 60, 30)  
Dataset 3  (51642.91448793635, 30, 29)  


#### **Comment:**  
In general, the smaller the initial is, the lower the acceptance rate of worse solution is. From test results, we can see that  
+) initial = 0.1 seems to be too large, the solver accepts much worse solutions and wastes time/efforts in them. Therefore, there isn't any improvement found in all 3 datasets in 2 times testing  
+) initial = 0.01 performs best among all settings in dataset 1. It gives fairly good result in dataset 2 but cannot give any improvement in dataset 3. On the other hand, initial = 0.001 works equally (to initial = 0.01) in dataset 2 and is the only setting that can give an improvement in the biggest dataset.  
The reason is for relatively smaller problem size as dataset 1, initial = 0.01 is less restricted and accepts much 'worse' solutions (compared to initial = 0.001). Thus it can jump out of the local optima and find new better solution while initial = 0.001 cannot. In the constrast, for a bigger problem size, because the solution space is larger, we need a lower temperature to restrict more 'worse' solutions to search more on focus.  


### **Comparision between Cooling schedules: Cooling factor**  
Run 3 settings of Cooling factor: 0.90, 0.95 and 0.99 with 3 datasets. The starting solution, neighborhood definition are chosen as successive insertion and two_opt, based on the previous comparisions. The initial temperature is set as 0.001\*initial_length to focus more on the dataset 2 and 3. The number of evaluation each iteration is default. As in the case of initial temperature test, to focus more on the effect of temperature, the termination condition is set as the maximum number of iterations = 70, 60, 30 for dataset 1, 2, 3 respectively. So for each dataset, the range of temperature explored in each setting case will be solely depends on the cooling factor and is the same in each time we test.

In [51]:
import pandas as pd
value = pd.DataFrame(index = index, columns = ['cooling factor = 0.90', 'cooling factor = 0.95', 'cooling factor = 0.99'])
factor_set = [0.90, 0.95, 0.99]
iter_set = [70, 60, 30]

for i in range(3): #dataset 
    for j in range(3): #factor
        result = SA (start[i][0], start[i][1], dataset[i][1], initial=0.001, 
                     cool_factor=factor_set[j], eval_times=10000, neighbor = 'two_opt', max_iteration = iter_set[i])
        value.iloc[i,j] = result

[0, 60, 15, 59, 23, 110, 95, 89, 53, 7, 69, 115, 33, 25, 70, 46, 88, 111, 35, 34, 9, 98, 105, 66, 36, 61, 103, 83, 5, 54, 47, 109, 100, 101, 2, 81, 79, 52, 26, 4, 62, 113, 72, 56, 82, 38, 94, 96, 11, 76, 63, 87, 108, 20, 92, 1, 114, 10, 50, 12, 22, 8, 102, 118, 3, 39, 104, 71, 37, 6, 55, 40, 41, 97, 117, 48, 16, 49, 45, 19, 106, 112, 68, 64, 42, 67, 90, 78, 57, 99, 32, 51, 107, 24, 18, 17, 84, 116, 30, 65, 80, 93, 21, 85, 74, 43, 86, 73, 13, 77, 44, 27, 91, 31, 58, 14, 29, 119, 28, 75, 0]
7779.0
Reach max number of iteration
Terminate
[0, 75, 28, 119, 91, 31, 29, 58, 14, 80, 93, 77, 44, 27, 73, 86, 13, 43, 74, 85, 21, 65, 30, 116, 84, 17, 18, 24, 107, 42, 78, 51, 32, 99, 57, 90, 67, 64, 68, 112, 106, 19, 45, 49, 117, 48, 16, 97, 41, 40, 55, 6, 37, 71, 104, 39, 33, 3, 118, 102, 8, 22, 12, 50, 10, 114, 1, 92, 20, 108, 87, 96, 11, 94, 63, 76, 38, 62, 4, 79, 26, 52, 81, 2, 101, 100, 47, 109, 111, 105, 113, 72, 56, 82, 66, 36, 61, 98, 9, 34, 103, 35, 83, 5, 88, 54, 46, 70, 25, 69, 115, 7, 5

Reach max number of iteration
Terminate
[0, 40, 41, 42, 43, 44, 45, 48, 47, 46, 51, 50, 49, 55, 52, 53, 54, 126, 125, 127, 124, 123, 122, 121, 57, 56, 58, 39, 37, 36, 60, 59, 120, 119, 118, 128, 148, 154, 155, 149, 153, 152, 151, 150, 230, 229, 228, 233, 231, 232, 235, 234, 227, 226, 156, 157, 158, 159, 225, 160, 161, 147, 146, 145, 144, 164, 163, 162, 224, 223, 237, 236, 263, 239, 238, 222, 221, 167, 166, 165, 143, 142, 170, 169, 168, 220, 219, 241, 240, 243, 242, 218, 217, 173, 172, 171, 141, 140, 174, 175, 176, 215, 216, 244, 245, 246, 247, 213, 214, 177, 178, 179, 139, 138, 180, 181, 182, 211, 212, 248, 249, 250, 251, 209, 210, 183, 184, 185, 137, 136, 186, 187, 188, 207, 208, 252, 253, 262, 261, 260, 254, 255, 205, 206, 189, 257, 256, 203, 204, 192, 193, 194, 190, 191, 135, 134, 133, 195, 196, 197, 201, 202, 258, 259, 199, 200, 198, 132, 131, 130, 129, 100, 101, 102, 71, 72, 18, 17, 73, 74, 97, 98, 99, 94, 95, 96, 75, 76, 12, 77, 78, 91, 92, 93, 88, 89, 90, 79, 85, 86, 87, 84, 83,

In [44]:
value7=value
print(value7)

                 cooling factor = 0.90         cooling factor = 0.95  \
Dataset 1             (7387.0, 70, 22)              (7310.0, 70, 18)   
Dataset 2  (76050.00479508081, 60, 19)   (74680.05091800306, 60, 34)   
Dataset 3  (53295.34844141955, 30, 23)  (52598.228465252265, 30, 29)   

                 cooling factor = 0.99  
Dataset 1             (7358.0, 70, 46)  
Dataset 2  (75650.53220616734, 60, 34)  
Dataset 3  (51814.98620027635, 30, 28)  


In [53]:
value8=value
print(value8)

                 cooling factor = 0.90        cooling factor = 0.95  \
Dataset 1             (7343.0, 70, 12)             (7318.0, 70, 10)   
Dataset 2  (74975.26034247996, 60, 27)  (74944.46545975405, 60, 26)   
Dataset 3  (52518.72908545497, 30, 29)  (51181.89991688918, 30, 19)   

                 cooling factor = 0.99  
Dataset 1             (7355.0, 70, 44)  
Dataset 2  (75854.69577156549, 60, 53)  
Dataset 3  (54466.10599868843, 30, 30)  


#### Comment:
In general, the smaller the cooling factor is, the faster the reduction speed of acceptance rate is. From test results, we can see that cooling factor = 0.95 setting gives the best value in general, which suggests that this reduction speed is not too slow and thus wasting time on 'very worse' solutions nor the speed is not too fast and thus quickly ending up at the local optima and cannot jump out. The fact that the cooling factor = 0.9 generally finds its best solution earlier (needing fewer iterations) than the other settings also proves the above statement.

### **Comparision between Cooling schedules: Number of evaluations at a given temperature** 
Run 3 settings of Cooling factor: 0.90, 0.95 and 0.99 with 3 datasets. The starting solution, neighborhood definition are chosen as successive insertion and two_opt, based on the previous comparisions. The initial temperature is set as 0.001\*initial_length to focus more on the dataset 2 and 3. The number of evaluation each iteration is default. As in the case of initial temperature test, to focus more on the effect of temperature, the termination condition is set as the maximum number of iterations = 70, 60, 30 for dataset 1, 2, 3 respectively. So for each dataset, the range of temperature explored in each setting case will be solely depends on the cooling factor and is the same in each time we test.

In [56]:
import pandas as pd
value = pd.DataFrame(index = index, columns = ['eval = 1000', 'eval = 10000', 'eval = 50000'])
eval_set = [5000, 10000, 20000]
iter_set = [70, 60, 30]

for i in range(3): #dataset 
    for j in range(3): #evaluation time
        result = SA (start[i][0], start[i][1], dataset[i][1], initial=0.001, 
                     cool_factor=0.95, eval_times=eval_set[j], neighbor = 'two_opt', max_iteration = iter_set[i])
        value.iloc[i,j] = result

[0, 60, 15, 59, 23, 110, 95, 89, 53, 7, 69, 115, 33, 25, 70, 46, 88, 111, 35, 34, 9, 98, 105, 66, 36, 61, 103, 83, 5, 54, 47, 109, 100, 101, 2, 81, 79, 52, 26, 4, 62, 113, 72, 56, 82, 38, 94, 96, 11, 76, 63, 87, 108, 20, 92, 1, 114, 10, 50, 12, 22, 8, 102, 118, 3, 39, 104, 71, 37, 6, 55, 40, 41, 97, 117, 48, 16, 49, 45, 19, 106, 112, 68, 64, 42, 67, 90, 78, 57, 99, 32, 51, 107, 24, 18, 17, 84, 116, 30, 65, 80, 93, 21, 85, 74, 43, 86, 73, 13, 77, 44, 27, 91, 31, 58, 14, 29, 119, 28, 75, 0]
7779.0
Reach max number of iteration
Terminate
[0, 75, 58, 14, 29, 28, 119, 31, 91, 27, 44, 77, 80, 93, 85, 13, 73, 86, 43, 74, 21, 65, 30, 116, 84, 17, 18, 24, 107, 42, 78, 51, 32, 99, 57, 90, 67, 64, 68, 112, 106, 19, 45, 49, 117, 48, 16, 97, 41, 40, 55, 6, 37, 71, 104, 39, 33, 3, 118, 102, 8, 22, 12, 50, 10, 114, 1, 81, 2, 79, 100, 101, 47, 109, 88, 35, 111, 105, 113, 72, 62, 4, 26, 52, 92, 20, 108, 87, 96, 11, 94, 63, 76, 38, 82, 56, 66, 36, 61, 98, 103, 9, 34, 83, 5, 54, 46, 70, 25, 69, 115, 7, 5

Reach max number of iteration
Terminate
[0, 40, 41, 42, 43, 44, 45, 48, 47, 46, 51, 50, 49, 55, 52, 53, 54, 126, 125, 127, 124, 123, 122, 121, 57, 56, 58, 39, 37, 36, 60, 59, 120, 119, 118, 116, 117, 128, 149, 153, 152, 151, 150, 230, 229, 228, 233, 231, 232, 235, 234, 227, 226, 156, 155, 154, 148, 157, 158, 159, 225, 160, 161, 147, 145, 146, 144, 164, 163, 162, 224, 223, 237, 236, 263, 239, 238, 222, 221, 167, 166, 165, 143, 142, 170, 169, 168, 220, 219, 241, 240, 243, 242, 218, 217, 173, 172, 171, 141, 140, 174, 175, 176, 215, 216, 244, 245, 246, 247, 213, 214, 177, 178, 179, 139, 138, 180, 181, 182, 211, 212, 248, 249, 250, 251, 209, 210, 183, 184, 185, 137, 136, 186, 187, 188, 207, 208, 252, 253, 262, 261, 260, 254, 255, 205, 206, 257, 256, 203, 204, 192, 189, 190, 191, 135, 194, 193, 196, 197, 201, 202, 258, 259, 199, 200, 198, 132, 195, 133, 134, 131, 130, 129, 94, 95, 96, 75, 76, 12, 77, 78, 91, 92, 93, 88, 89, 90, 79, 80, 82, 85, 86, 87, 84, 83, 81, 5, 1, 2, 3, 4, 8, 7, 6, 11, 

In [58]:
value9=value
print(value9)

                           eval = 1000                 eval = 10000  \
Dataset 1             (7360.0, 70, 29)             (7366.0, 70, 10)   
Dataset 2  (78281.91942828834, 60, 47)  (77100.43398597579, 60, 53)   
Dataset 3  (53140.31764817589, 30, 30)  (52247.59795011435, 30, 23)   

                          eval = 50000  
Dataset 1             (7375.0, 70, 15)  
Dataset 2  (75281.51612826344, 60, 52)  
Dataset 3  (49649.79619740871, 30, 27)  


#### Comment:
The test results show that more evaluation times in each iteration gives better objective value in all dataset.

### **Comparision between Termination criteria: TimeLimit / Max number of iterations** 
In general, both TimeLimit (the time duration the function run) and Max number of iterations both restricts the number of times we change (cooling down) the temperature. Therefore, we will test only max number of iterations here.   
Run 3 settings of max number of iterations: \[50, 70, 90\], \[30, 60, 90\] and \[10, 30, 50\] with dataset 1, 2, 3 respectively. All parameters are set as in previous tests.

In [61]:
import pandas as pd
value = pd.DataFrame(index = index, columns = ['max_iteration = 50/30/10', 'max_iteration = 70/60/30', 'max_iteration = 90/90/50'])
iter_set = [[50, 70, 90], [30, 60, 90], [10, 30, 50]]

for i in range(3): #dataset 
    for j in range(3): #evaluation time
        result = SA (start[i][0], start[i][1], dataset[i][1], initial=0.001, 
                     cool_factor=0.95, eval_times=10000, neighbor = 'two_opt', max_iteration = iter_set[i][j])
        value.iloc[i,j] = result

[0, 60, 15, 59, 23, 110, 95, 89, 53, 7, 69, 115, 33, 25, 70, 46, 88, 111, 35, 34, 9, 98, 105, 66, 36, 61, 103, 83, 5, 54, 47, 109, 100, 101, 2, 81, 79, 52, 26, 4, 62, 113, 72, 56, 82, 38, 94, 96, 11, 76, 63, 87, 108, 20, 92, 1, 114, 10, 50, 12, 22, 8, 102, 118, 3, 39, 104, 71, 37, 6, 55, 40, 41, 97, 117, 48, 16, 49, 45, 19, 106, 112, 68, 64, 42, 67, 90, 78, 57, 99, 32, 51, 107, 24, 18, 17, 84, 116, 30, 65, 80, 93, 21, 85, 74, 43, 86, 73, 13, 77, 44, 27, 91, 31, 58, 14, 29, 119, 28, 75, 0]
7779.0
Reach max number of iteration
Terminate
[0, 60, 15, 59, 23, 110, 95, 89, 53, 7, 69, 25, 70, 46, 54, 5, 83, 34, 9, 103, 98, 61, 36, 66, 82, 56, 72, 113, 105, 111, 35, 88, 109, 47, 101, 100, 79, 2, 81, 52, 26, 4, 62, 38, 76, 63, 94, 11, 96, 87, 108, 20, 92, 1, 114, 10, 50, 12, 22, 8, 102, 118, 3, 33, 115, 104, 39, 71, 37, 6, 55, 40, 41, 97, 117, 48, 16, 49, 45, 19, 106, 112, 68, 64, 67, 90, 57, 99, 32, 51, 78, 42, 107, 24, 18, 17, 84, 116, 30, 65, 21, 80, 93, 85, 13, 74, 43, 86, 73, 77, 44, 27, 9

Reach max number of iteration
Terminate
[0, 38, 35, 34, 33, 32, 31, 29, 28, 27, 26, 21, 20, 19, 15, 14, 9, 10, 7, 6, 80, 8, 3, 2, 1, 4, 5, 81, 83, 84, 87, 86, 85, 82, 79, 90, 89, 88, 93, 92, 91, 78, 77, 11, 13, 12, 76, 75, 96, 95, 94, 131, 132, 198, 133, 195, 196, 197, 201, 200, 199, 259, 258, 202, 193, 194, 134, 135, 191, 190, 192, 204, 203, 257, 256, 261, 260, 262, 254, 255, 205, 189, 206, 208, 253, 252, 207, 188, 187, 136, 186, 137, 185, 184, 183, 210, 209, 251, 250, 249, 248, 212, 211, 182, 181, 180, 138, 139, 179, 178, 177, 214, 213, 247, 246, 245, 244, 216, 215, 176, 175, 174, 140, 141, 171, 172, 173, 217, 218, 242, 243, 240, 241, 219, 220, 169, 168, 170, 142, 143, 165, 166, 167, 221, 222, 238, 239, 263, 236, 237, 223, 224, 162, 163, 164, 144, 146, 145, 147, 161, 160, 225, 159, 158, 157, 148, 154, 155, 156, 226, 227, 234, 235, 232, 231, 233, 228, 229, 230, 150, 151, 152, 153, 149, 130, 129, 99, 98, 74, 97, 73, 17, 16, 18, 72, 71, 102, 101, 100, 105, 104, 103, 70, 69, 23, 22, 25, 

In [63]:
value10=value
print(value10)

              max_iteration = 50/30/10     max_iteration = 70/60/30  \
Dataset 1             (7342.0, 50, 10)             (7374.0, 70, 15)   
Dataset 2  (76509.78049737819, 30, 27)  (77362.42888549241, 60, 23)   
Dataset 3  (56609.429419290915, 10, 4)  (51763.29948290596, 30, 28)   

               max_iteration = 90/90/50  
Dataset 1              (7367.0, 90, 10)  
Dataset 2     (74968.184941172, 90, 40)  
Dataset 3  (50929.062234521196, 50, 40)  


#### Comment:
In general, the more iterations run, the better the objective is, especially for a large size problem with large solution space leading to more space to improve. In the relatively smaller dataset, the improvement when increasing max_iteration is not big and the best solution is found in early iteration.

### **Comparision between Termination criteria: Max number of iteration without improvement** 
Run 3 settings of max number of iterations without improvement: non_improvement = 5, 10, 15 with all datasets. All parameters are set as in previous tests.

In [69]:
import pandas as pd
value = pd.DataFrame(index = index, columns = ['non_improvement = 5', 'non_improvement = 10', 'non_improvement = 15'])
imp_set = [5, 10, 15]

for i in range(3): #dataset 
    for j in range(3): #non_improvement iteration
        result = SA (start[i][0], start[i][1], dataset[i][1], initial=0.001, 
                     cool_factor=0.95, eval_times=10000, neighbor = 'two_opt', 
                     non_improvement = imp_set[j])
        value.iloc[i,j] = result

[0, 60, 15, 59, 23, 110, 95, 89, 53, 7, 69, 115, 33, 25, 70, 46, 88, 111, 35, 34, 9, 98, 105, 66, 36, 61, 103, 83, 5, 54, 47, 109, 100, 101, 2, 81, 79, 52, 26, 4, 62, 113, 72, 56, 82, 38, 94, 96, 11, 76, 63, 87, 108, 20, 92, 1, 114, 10, 50, 12, 22, 8, 102, 118, 3, 39, 104, 71, 37, 6, 55, 40, 41, 97, 117, 48, 16, 49, 45, 19, 106, 112, 68, 64, 42, 67, 90, 78, 57, 99, 32, 51, 107, 24, 18, 17, 84, 116, 30, 65, 80, 93, 21, 85, 74, 43, 86, 73, 13, 77, 44, 27, 91, 31, 58, 14, 29, 119, 28, 75, 0]
7779.0
Reach max number of iteration without new best solution
Terminate
[0, 60, 15, 59, 23, 110, 95, 89, 53, 7, 115, 69, 25, 70, 46, 47, 88, 54, 5, 83, 34, 9, 98, 61, 36, 66, 105, 103, 35, 111, 109, 100, 101, 2, 81, 52, 4, 26, 79, 113, 72, 56, 82, 38, 62, 76, 63, 94, 11, 96, 87, 108, 20, 92, 1, 114, 10, 50, 12, 22, 8, 102, 118, 3, 33, 39, 104, 71, 37, 6, 55, 40, 41, 97, 48, 117, 16, 49, 45, 19, 106, 112, 68, 64, 67, 90, 57, 99, 32, 51, 78, 42, 107, 24, 18, 17, 84, 116, 30, 65, 21, 80, 93, 85, 13, 74,

Reach max number of iteration without new best solution
Terminate
[0, 38, 33, 32, 31, 27, 25, 26, 21, 19, 20, 15, 13, 14, 9, 7, 8, 3, 2, 1, 4, 5, 81, 83, 84, 87, 86, 85, 82, 6, 80, 79, 90, 89, 88, 93, 92, 91, 78, 77, 11, 10, 12, 76, 75, 96, 95, 94, 131, 132, 198, 133, 195, 196, 197, 200, 199, 259, 258, 202, 201, 194, 134, 135, 191, 190, 193, 192, 204, 203, 257, 256, 205, 206, 189, 255, 254, 260, 261, 262, 253, 252, 208, 207, 188, 187, 186, 136, 137, 185, 184, 183, 210, 209, 251, 250, 249, 248, 212, 211, 182, 181, 180, 138, 139, 179, 178, 177, 214, 213, 247, 246, 245, 244, 216, 215, 176, 175, 174, 140, 141, 171, 172, 173, 217, 218, 242, 243, 240, 241, 219, 220, 170, 142, 143, 165, 166, 169, 168, 167, 221, 222, 238, 239, 263, 236, 237, 223, 224, 162, 163, 164, 144, 146, 147, 145, 161, 160, 225, 159, 158, 157, 148, 154, 155, 156, 226, 227, 234, 235, 232, 231, 233, 228, 229, 152, 151, 230, 150, 153, 149, 130, 129, 99, 98, 97, 74, 73, 17, 16, 18, 72, 71, 102, 101, 100, 105, 104, 103, 70, 69

In [70]:
value11=value
print(value11)

                   non_improvement = 5          non_improvement = 10  \
Dataset 1              (7337.0, 14, 9)               (7351.0, 15, 5)   
Dataset 2  (74604.39530827226, 32, 27)    (75712.8895799506, 63, 53)   
Dataset 3    (57260.05492393449, 5, 0)  (52744.293439486595, 46, 36)   

                  non_improvement = 15  
Dataset 1             (7361.0, 25, 10)  
Dataset 2  (75027.55289218997, 65, 50)  
Dataset 3  (52266.30288376697, 74, 59)  


#### Comment:
Similar to the previous test, the more the allowed number of non-improvement run is, the more iterations run and thus the better the objective is, especially for a large size problem (dataset 3). In large dataset, it takes more iterations to jump out of the local optima, so higher number of non_improvement runs accepted should be used.  
Compared with the termination strategy of running time/iteration, this strategy (using max number of non-improvement iteration) is harder to use and predicted because it depends more on the random selection of neighborhood.